In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Advanced data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    rescale=1./255
)

# Enhanced RICAP or CutMix for better augmentation
def cutmix_augmentation(images, labels, alpha=1.0):
    batch_size = tf.shape(images)[0]
    lam = tf.random.uniform([batch_size], 0, 1)
    
    # Create mixed images and labels
    indices = tf.random.shuffle(tf.range(batch_size))
    shuffled_images = tf.gather(images, indices)
    shuffled_labels = tf.gather(labels, indices)
    
    return mixed_images, mixed_labels


In [24]:
import tensorflow as tf
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import os

# 1. Calculate class weights (unchanged)
def calculate_class_weights(y_train):
    if y_train.ndim > 1:
        y_labels = np.argmax(y_train, axis=1)
    else:
        y_labels = y_train
    cw = compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
    return dict(enumerate(cw))

# 2. Build tf.data.Dataset with oversampling
def make_balanced_dataset(image_paths, labels, batch_size=32):
    # Convert to Dataset
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Separate by class
    class_datasets = {}
    for cls in np.unique(labels):
        cls_ds = ds.filter(lambda img, lbl, c=cls: tf.equal(lbl, c))
        class_datasets[int(cls)] = cls_ds.repeat()  # infinite repetition
    
    # Compute class counts to set sampling weights
    counts = np.bincount(labels)
    sampling_weights = counts.sum() / (counts * len(counts))
    
    # Create a sampling dataset
    choices = list(class_datasets.keys())
    prob_ds = tf.data.Dataset.from_tensor_slices(sampling_weights).repeat()
    
    # Sample classes according to weights
    balanced_ds = tf.data.experimental.sample_from_datasets(
        [class_datasets[c] for c in choices],
        weights=sampling_weights.tolist()
    )
    
    # Preprocess and batch
    def preprocess(img_path, lbl):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224]) / 255.0
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.2)
        return img, lbl
    
    return (balanced_ds
            .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .shuffle(1000)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

# 3. Corrected focal loss (from previous code)
def focal_loss(alpha=0.25, gamma=2.0):
    def loss_fn(y_true, y_pred):
        y_pred = tf.nn.softmax(y_pred, axis=-1)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        if y_true.ndim == 1 or y_true.shape[-1] == 1:
            y_true_ohe = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])
        else:
            y_true_ohe = y_true
        cross_entropy = -y_true_ohe * tf.math.log(y_pred)
        p_t = tf.reduce_sum(y_true_ohe * y_pred, axis=-1, keepdims=True)
        weight = alpha * tf.pow(1 - p_t, gamma)
        return tf.reduce_mean(weight * tf.reduce_sum(cross_entropy, axis=-1))
    return loss_fn

def load_image_paths_and_labels(base_dir):
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(base_dir))   # ensure consistent class order
    class_indices = {cls: idx for idx, cls in enumerate(class_names)}
    
    for cls in class_names:
        cls_dir = os.path.join(base_dir, cls)
        for fname in os.listdir(cls_dir):
            fpath = os.path.join(cls_dir, fname)
            image_paths.append(fpath)
            labels.append(class_indices[cls])
    
    return image_paths, labels, class_names

# Build datasets
train_paths, train_labels, class_names = load_image_paths_and_labels(r"C:\Users\ASUS\OneDrive\Desktop\ML\datasets_split\Train")
val_paths,   val_labels,   _           = load_image_paths_and_labels(r"C:\Users\ASUS\OneDrive\Desktop\ML\datasets_split\val")
test_paths,  test_labels,  _           = load_image_paths_and_labels(r"C:\Users\ASUS\OneDrive\Desktop\ML\datasets_split\Test")

print(f"Classes: {class_names}")
print(f"Train samples: {len(train_paths)}, Val samples: {len(val_paths)}, Test samples: {len(test_paths)}")


Classes: ['coffee___healthy', 'coffee___rust', 'coffee__phoma', 'corn_blight', 'corn_common_rust', 'corn_healthy', 'cotton_bacterial_blight', 'cotton_curl_virus', 'cotton_healthy_leaf', 'cotton_herbicide_growth_damage', 'cotton_leaf_hopper_jassids', 'cotton_leaf_redding', 'cotton_leaf_variegation', 'potato___early_blight', 'potato___healthy', 'potato___late_blight', 'rice_bacterialblight', 'rice_brown_spot', 'rice_healthy', 'rice_leafsmut']
Train samples: 19479, Val samples: 2902, Test samples: 2481


In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, Dense

def create_improved_cnn(input_shape, num_classes):
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        GlobalAveragePooling2D(),
        Dropout(0.5),
        
        # Classification layer
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model


In [26]:
from tensorflow.keras.applications import EfficientNetB0, ResNet50, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

def create_transfer_learning_model(base_model_name='EfficientNetB0', input_shape=(224, 224, 3), num_classes=23):
    # Load pre-trained model
    if base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Add custom classification head
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.3),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Fine-tuning strategy
def fine_tune_model(model, base_model, learning_rate=1e-5):
    # Unfreeze top layers for fine-tuning
    base_model.trainable = True
    
    # Fine-tune from this layer onwards
    fine_tune_at = len(base_model.layers) - 20
    
    # Freeze all layers except the top ones
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Use lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate/10),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [30]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Early stopping and learning rate reduction
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=8,
    min_lr=1e-7,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Compile model with appropriate optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=focal_loss(gamma=2.0, alpha=0.25),  # Focal loss for imbalance
    metrics=['accuracy']
)

# Training with callbacks (use train_ds, val_ds)
history = model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds,
    # ⚠️ Remove class_weight if using oversampling
    # class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)


Epoch 1/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8771 - loss: 0.4561
Epoch 1: val_accuracy improved from None to 0.87500, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 967s 2s/step - accuracy: 0.8787 - loss: 0.4545 - val_accuracy: 0.8750 - val_loss: 0.4546 - learning_rate: 0.0010
Epoch 2/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.8877 - loss: 0.4493
Epoch 2: val_accuracy did not improve from 0.87500
628/628 ━━━━━━━━━━━━━━━━━━━━ 512s 815ms/step - accuracy: 0.8898 - loss: 0.4489 - val_accuracy: 0.8730 - val_loss: 0.4530 - learning_rate: 0.0010
Epoch 3/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 673ms/step - accuracy: 0.8978 - loss: 0.4463
Epoch 3: val_accuracy improved from 0.87500 to 0.87997, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 483s 769ms/step - accuracy: 0.8958 - loss: 0.4464 - val_accuracy: 0.8800 - val_loss: 0.4495 - learning_rate: 0.0010
Epoch 4/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 616ms/step - accuracy: 0.8933 - loss: 0.4456
Epoch 4: val_accuracy did not improve from 0.87997
628/628 ━━━━━━━━━━━━━━━━━━━━ 444s 707ms/step - accuracy: 0.8978 - loss: 0.4446 - val_accuracy: 0.8757 - val_loss: 0.4490 - learning_rate: 0.0010
Epoch 5/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 674ms/step - accuracy: 0.8996 - loss: 0.4436
Epoch 5: val_accuracy improved from 0.87997 to 0.88793, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 484s 770ms/step - accuracy: 0.9029 - loss: 0.4429 - val_accuracy: 0.8879 - val_loss: 0.4467 - learning_rate: 0.0010
Epoch 6/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 689ms/step - accuracy: 0.9058 - loss: 0.4417
Epoch 6: val_accuracy did not improve from 0.88793
628/628 ━━━━━━━━━━━━━━━━━━━━ 492s 783ms/step - accuracy: 0.9058 - loss: 0.4417 - val_accuracy: 0.8849 - val_loss: 0.4461 - learning_rate: 0.0010
Epoch 7/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 690ms/step - accuracy: 0.9083 - loss: 0.4411
Epoch 7: val_accuracy did not improve from 0.88793
628/628 ━━━━━━━━━━━━━━━━━━━━ 492s 783ms/step - accuracy: 0.9091 - loss: 0.4407 - val_accuracy: 0.8876 - val_loss: 0.4465 - learning_rate: 0.0010
Epoch 8/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.9070 - loss: 0.4408
Epoch 8: val_accuracy did not improve from 0.88793
628/628 ━━━━━━━━━━━━━━━━━━━━ 446s 710ms/step - accuracy: 0.9073 - loss: 0.4405 - val_accuracy: 0.8846 - val_loss: 0.4467 - learning_rate: 0.0010


628/628 ━━━━━━━━━━━━━━━━━━━━ 387s 616ms/step - accuracy: 0.9100 - loss: 0.4398 - val_accuracy: 0.8896 - val_loss: 0.4454 - learning_rate: 0.0010
Epoch 10/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 540ms/step - accuracy: 0.9076 - loss: 0.4400
Epoch 10: val_accuracy did not improve from 0.88959
628/628 ━━━━━━━━━━━━━━━━━━━━ 387s 617ms/step - accuracy: 0.9086 - loss: 0.4398 - val_accuracy: 0.8896 - val_loss: 0.4445 - learning_rate: 0.0010
Epoch 11/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - accuracy: 0.9117 - loss: 0.4390
Epoch 11: val_accuracy did not improve from 0.88959
628/628 ━━━━━━━━━━━━━━━━━━━━ 390s 622ms/step - accuracy: 0.9101 - loss: 0.4392 - val_accuracy: 0.8846 - val_loss: 0.4462 - learning_rate: 0.0010
Epoch 12/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 561ms/step - accuracy: 0.9162 - loss: 0.4379
Epoch 12: val_accuracy improved from 0.88959 to 0.89423, saving model to best_model.h5


...

628/628 ━━━━━━━━━━━━━━━━━━━━ 427s 679ms/step - accuracy: 0.9254 - loss: 0.4330 - val_accuracy: 0.9125 - val_loss: 0.4371 - learning_rate: 2.0000e-04
Epoch 49/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 595ms/step - accuracy: 0.9296 - loss: 0.4321
Epoch 49: val_accuracy improved from 0.91247 to 0.91578, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 427s 681ms/step - accuracy: 0.9286 - loss: 0.4324 - val_accuracy: 0.9158 - val_loss: 0.4365 - learning_rate: 2.0000e-04
Epoch 50/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.9293 - loss: 0.4320
Epoch 50: val_accuracy did not improve from 0.91578
628/628 ━━━━━━━━━━━━━━━━━━━━ 424s 675ms/step - accuracy: 0.9276 - loss: 0.4324 - val_accuracy: 0.9151 - val_loss: 0.4366 - learning_rate: 2.0000e-04
Epoch 51/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.9244 - loss: 0.4338
Epoch 51: val_accuracy did not improve from 0.91578
628/628 ━━━━━━━━━━━━━━━━━━━━ 425s 677ms/step - accuracy: 0.9276 - loss: 0.4327 - val_accuracy: 0.9138 - val_loss: 0.4363 - learning_rate: 2.0000e-04
Epoch 52/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.9260 - loss: 0.4330
Epoch 52: val_accuracy did not improve from 0.91578
628/628 ━━━━━━━━━━━━━━━━━━━━ 425s 676ms/step - accuracy: 0.9278 - loss: 0.4325 - val_accuracy: 0.9125 - val_loss: 0.4369 - lear

628/628 ━━━━━━━━━━━━━━━━━━━━ 427s 681ms/step - accuracy: 0.9262 - loss: 0.4328 - val_accuracy: 0.9161 - val_loss: 0.4358 - learning_rate: 2.0000e-04
Epoch 56/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 599ms/step - accuracy: 0.9318 - loss: 0.4319
Epoch 56: val_accuracy improved from 0.91611 to 0.91810, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 431s 687ms/step - accuracy: 0.9294 - loss: 0.4323 - val_accuracy: 0.9181 - val_loss: 0.4354 - learning_rate: 2.0000e-04
Epoch 57/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 589ms/step - accuracy: 0.9295 - loss: 0.4319
Epoch 57: val_accuracy did not improve from 0.91810
628/628 ━━━━━━━━━━━━━━━━━━━━ 424s 675ms/step - accuracy: 0.9290 - loss: 0.4321 - val_accuracy: 0.9168 - val_loss: 0.4357 - learning_rate: 2.0000e-04
Epoch 58/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 591ms/step - accuracy: 0.9274 - loss: 0.4329
Epoch 58: val_accuracy did not improve from 0.91810
628/628 ━━━━━━━━━━━━━━━━━━━━ 425s 676ms/step - accuracy: 0.9283 - loss: 0.4324 - val_accuracy: 0.9151 - val_loss: 0.4361 - learning_rate: 2.0000e-04
Epoch 59/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 590ms/step - accuracy: 0.9319 - loss: 0.4315
Epoch 59: val_accuracy improved from 0.91810 to 0.91877, saving model to best_model.h5


628/628 ━━━━━━━━━━━━━━━━━━━━ 430s 684ms/step - accuracy: 0.9298 - loss: 0.4319 - val_accuracy: 0.9188 - val_loss: 0.4353 - learning_rate: 2.0000e-04
Epoch 60/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - accuracy: 0.9245 - loss: 0.4335
Epoch 60: val_accuracy did not improve from 0.91877
628/628 ━━━━━━━━━━━━━━━━━━━━ 428s 681ms/step - accuracy: 0.9260 - loss: 0.4329 - val_accuracy: 0.9158 - val_loss: 0.4357 - learning_rate: 2.0000e-04
Epoch 61/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 591ms/step - accuracy: 0.9303 - loss: 0.4319
Epoch 61: val_accuracy did not improve from 0.91877
628/628 ━━━━━━━━━━━━━━━━━━━━ 424s 675ms/step - accuracy: 0.9290 - loss: 0.4322 - val_accuracy: 0.9155 - val_loss: 0.4358 - learning_rate: 2.0000e-04
Epoch 62/100
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - accuracy: 0.9317 - loss: 0.4311
Epoch 62: val_accuracy did not improve from 0.91877
628/628 ━━━━━━━━━━━━━━━━━━━━ 426s 678ms/step - accuracy: 0.9315 - loss: 0.4314 - val_accuracy: 0.9125 - val_loss: 0.4364 - lear

In [69]:


import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import EfficientNetB0, ResNet50, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model, load_model

# --- 1. Data Preprocessing ---

def preprocess(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode images as RGB
    img = tf.image.resize(img, [256, 256])       # Resize all images to 256x256 (or 224x224 as per model)
    img = img / 255.0                             # Normalize pixel values to [0,1]
    return img, label

# Example of creating tf.data.Dataset for testing
def create_dataset(image_paths, labels, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# --- 2. Transfer Learning Model Creator ---

def create_transfer_learning_model(base_model_name, input_shape=(256, 256, 3), num_classes=23):
    # Enforce 3 channels for pretrained model weights compatibility
    if input_shape[-1] != 3:
        raise ValueError(f"Input shape must have 3 channels for pretrained ImageNet weights, got {input_shape[-1]}")

    # Select base model with pretrained weights
    if base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Unknown base model name: {base_model_name}")

    # Freeze base model layers initially
    base_model.trainable = False

    # Add classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model, base_model

# --- 3. Ensemble Prediction Function ---

def create_ensemble_predictions(models, X_test):
    all_predictions = []
    for model in models:
        pred = model.predict(X_test)
        all_predictions.append(pred)
    # Average the predictions from all models
    ensemble_pred_probs = np.mean(all_predictions, axis=0)
    # Final class is argmax of averaged probabilities
    final_predictions = np.argmax(ensemble_pred_probs, axis=1)
    return final_predictions, ensemble_pred_probs

# --- 4. Example Usage ---

if __name__ == "__main__":
    # Assume train_paths, train_labels, val_paths, val_labels, test_paths, test_labels are defined and loaded externally
    batch_size = 32
    num_classes = 23  # Set as required for your dataset
    
    # Create dataset pipelines
    train_ds = create_dataset(train_paths, train_labels, batch_size=batch_size)
    val_ds = create_dataset(val_paths, val_labels, batch_size=batch_size)
    test_ds = create_dataset(test_paths, test_labels, batch_size=batch_size)

    architectures = ['EfficientNetB0', 'ResNet50', 'VGG16']
    models = []

    for arch in architectures:
        print(f"--- Training {arch} ---")
        model, base_model = create_transfer_learning_model(arch, input_shape=(256, 256, 3), num_classes=num_classes)
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        # Train your model here
        model.fit(train_ds, validation_data=val_ds, epochs=10)  # Adjust epochs as needed
        # Save the trained model
        model.save(f"{arch}_model.h5")
        models.append(model)

    # For inference, load test images as numpy arrays for ensemble prediction
    # Here we create a NumPy batch for demonstration
    X_test = np.array([img.numpy() for img, _ in test_ds.unbatch().take(100)])  # Take 100 samples, adjust accordingly

    # Ensemble predictions
    final_preds, ensemble_probs = create_ensemble_predictions(models, X_test)

    print("Ensemble final predicted classes:")
    print(final_preds)


--- Training EfficientNetB0 ---


ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)

In [51]:
model.save("plant_disease_modelp1.keras")   # new format


In [54]:
print(model.input_shape)

(None, 256, 256, 3)


In [61]:
for images, labels in train_ds.take(1):
    print(images.shape)  # Should print (batch_size, 224, 224, 3)


(32, 224, 224, 3)


In [12]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to test image
img_path = r"C:\Users\ASUS\OneDrive\画像\Screenshots\Screenshot 2025-09-16 210119.png"

# Preprocess the image (resize to 224x224 to match MobileNetV2)
img = image.load_img(img_path, target_size=(224, 224))  
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # normalize like training

# Make prediction
predictions = model.predict(img_array)

# Get predicted class
predicted_class = class_names[np.argmax(predictions)]

# Suggest medicine
medicine = disease_medicine_map.get(predicted_class, "No suggestion available")

print("Predicted class:", predicted_class)
print("Suggested medicine:", medicine)

# 🔎 Also print top-3 probabilities for debugging
probs = predictions[0]
top_indices = probs.argsort()[-3:][::-1]
print("\nTop 3 Predictions:")
for i in top_indices:
    print(f"{class_names[i]}: {probs[i]*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: coffee___healthy
Suggested medicine: No suggestion available

Top 3 Predictions:
coffee___healthy: 34.08%
coffee__phoma: 19.94%
corn_healthy: 10.87%


In [13]:
print(train_ds.class_names)


AttributeError: '_PrefetchDataset' object has no attribute 'class_names'

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# -----------------------------
# Load the correct model
# -----------------------------
model_path = r"C:\Users\ASUS\OneDrive\Desktop\ML\plant_disease_modelp1.keras"
model = load_model(model_path)

print(f"\n✅ Loaded model from: {model_path}")
model.summary()

# -----------------------------
# Define Classes (order must match training)
# -----------------------------
class_names = [
    "coffee___healthy",
    "coffee___rust",
    "coffee___phoma",
    "cotton___healthy",
    "cotton___bacterial_blight",
    "cotton___curl_virus",
    "maize___healthy",
    "maize___leaf_blight",
    "potato___healthy",
    "potato___early_blight",
    "rice___healthy",
    "rice___blast"
]

# -----------------------------
# Path to test image
# -----------------------------
img_path = r"C:\Users\ASUS\OneDrive\画像\Screenshots\Screenshot 2025-09-16 210119.png"

# Preprocess the image (resize to 224x224 for MobileNetV2)
img = image.load_img(img_path, target_size=(224, 224))  
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # normalize

# -----------------------------
# Make prediction
# -----------------------------
predictions = model.predict(img_array)

# Get predicted class
predicted_class = class_names[np.argmax(predictions)]
print("\nPredicted class:", predicted_class)

# -----------------------------
# Top-3 Predictions
# -----------------------------
probs = predictions[0]
top_indices = probs.argsort()[-3:][::-1]
print("\nTop 3 Predictions:")
for i in top_indices:
    print(f"{class_names[i]}: {probs[i]*100:.2f}%")


TypeError: Could not locate function 'loss_fn'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'loss_fn', 'registered_name': 'function'}

In [1]:
probs = predictions[0]
for i, p in enumerate(probs):
    print(f"{class_names[i]}: {p:.4f}")


NameError: name 'predictions' is not defined

In [16]:
import os
base_dir = "datasets_split/Train"
for cls in os.listdir(base_dir):
    print(cls, len(os.listdir(os.path.join(base_dir, cls))))


coffee__phoma 1000
coffee___healthy 973
coffee___rust 1000
corn_blight 910
corn_common_rust 1049
corn_gray_leaf_spot 401
corn_healthy 923
cotton_bacterial_blight 1000
cotton_curl_virus 1000
cotton_healthy_leaf 1000
cotton_herbicide_growth_damage 1000
cotton_leaf_hopper_jassids 970
cotton_leaf_redding 1104
cotton_leaf_variegation 781
potato___early_blight 1000
potato___healthy 922
potato___late_blight 1000
rice_bacterialblight 1000
rice_brown_spot 1000
rice_healthy 1041
rice_leafsmut 1000
